# 🧠 Advanced Agent Architectures: The ReAct Pattern

This notebook explores the ReAct (Reason + Act) pattern, the industry standard for building reliable autonomous agents. Unlike simple agents, a ReAct agent explicitly "thinks" before it acts and observes the results of its actions to decide its next move.

## 🎯 Learning Objectives

* **The Manual Loop:** Understand the mechanics of the Think-Act-Observe cycle.
* **Reasoning Extraction:** Learn how to capture and display the agent's internal thought process.
* **Safety & Control:** Implement iteration limits to prevent infinite loops and manage costs.
* **Autonomous Problem Solving:** Observe how the agent breaks down complex, multi-step user requests.

In [ ]:
%pip install openai python-dotenv

In [ ]:
# Setup with z.ai
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables
load_dotenv()

# Initialize OpenAI client (points to z.ai)
client = OpenAI(
    base_url=os.environ.get("ZAI_BASE_URL"),
    api_key=os.environ.get("ZAI_API_KEY")
)

MODEL = os.environ.get("ZAI_MODEL", "gpt-4o")

print("✅ ReAct Agent - Using LLM via z.ai")
print(f"🔗 Endpoint: {os.environ.get('ZAI_BASE_URL')}")
print(f"🤖 Model: {MODEL}")

In [ ]:
# Initialize Database
import importlib
import db_helper
importlib.reload(db_helper)
from db_helper import reset_database

reset_database()

## 🎯 THE REACT PATTERN

**ReAct = Reasoning + Acting**

This is the MOST IMPORTANT agent pattern. ~80% of production agents use this.

### The Loop:
```
┌──────────────────────────────────────────┐
│  1. THINK: What do I need?               │
│  2. ACT: Call a tool                     │
│  3. OBSERVE: Look at the result          │
│  4. THINK: Do I have enough info?        │
│     ├─ Yes → Answer the question         │
│     └─ No → Go back to step 2            │
└──────────────────────────────────────────┘
```

### Key Differences from Simple Agent:

❌ **Simple:** One tool call, done

✅ **ReAct:** Loop until satisfied

**Let's build it!**

In [ ]:
# ReAct Agent Implementation
from db_helper import lookup_customer, check_order_status, get_customer_orders, process_refund
from db_tools import tools
import json

def react_agent(user_message: str, max_iterations: int = 10):
    """
    An autonomous ReAct agent that manages its own reasoning loop.
    It continues to iterate until it determines the task is complete 
    or hits the max_iterations safety limit.
    """
    
    # The system prompt is the 'brain' that enforces the ReAct pattern
    system_prompt = """You are a helpful customer support assistant that uses tools.
                    Think step by step using this pattern:
                    1. What information do I need?
                    2. What tool can help me get that?
                    3. Use the tool and observe the result.
                    4. Do I have enough info to answer now?
                    
                    Always explain your reasoning before using tools. When finished, explain why you have enough information to stop.
                    Be thorough - if a customer asks for "all orders", make sure to get ALL their orders.
                    If you need to refund multiple items, do them one at a time."""
    
    # Initialize conversation with system prompt and user's message
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    print(f"🎯 TASK: {user_message}")
    print("="*80)
    
    # THE REACT LOOP
    for iteration in range(max_iterations):
        print(f"📍 ITERATION {iteration + 1}/{max_iterations}")
        print("-" * 80)
        
        # --- STEP 1: REASONING ---
        # The agent decides what to do based on the current history
        response = client.chat.completions.create(
            model=MODEL,
            max_tokens=2048,
            messages=messages,
            tools=tools
        )
        
        assistant_message = response.choices[0].message
        
        # Display the "Thought" process (content or reasoning_content for z.ai)
        thought = assistant_message.content or ""
        reasoning = getattr(assistant_message, 'reasoning_content', None)
        if reasoning:
            print(f"🤔 REASONING: {reasoning}")
        if thought:
            print(f"🤔 THOUGHT: {thought}")
        
        # --- STEP 2: CHECK COMPLETION ---
        # If the model doesn't want to use a tool, it's finished
        if response.choices[0].finish_reason != "tool_calls":
            final_answer = assistant_message.content or ""
            print("="*80)
            print(f"✅ FINAL ANSWER:\n{final_answer}")
            return final_answer
        
        # Record the agent's decision in history
        messages.append(assistant_message)
        
        # --- STEP 3: ACTION ---
        # Execute the tool calls requested by the agent
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_input = json.loads(tool_call.function.arguments)
            
            print(f"🔧 ACTION: {tool_name}")
            print(f"   Input: {json.dumps(tool_input, indent=2)}")
            
            # Routing logic: Map tool names to actual Python functions
            if tool_name == "lookup_customer":
                result = lookup_customer(**tool_input)
            elif tool_name == "check_order_status":
                result = check_order_status(**tool_input)
            elif tool_name == "get_customer_orders":
                result = get_customer_orders(**tool_input)
            elif tool_name == "process_refund":
                result = process_refund(**tool_input)
            else:
                result = {"error": f"Unknown function: {tool_name}"}
            
            print(f"📊 OBSERVATION: {json.dumps(result, indent=2)}\n")
            
            # --- STEP 4: FEEDBACK ---
            # Provide the observation back to the agent for the next iteration
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result)
            })
    
    print("⚠️ Max iterations reached - task may be incomplete")
    return "Task incomplete - reached maximum iterations"

## 🧪 Testing Scenarios

### 1. Simple Single-Step Task

This test confirms that the agent can handle a basic request in a single iteration. Since all the information (Order ID) is provided, the agent should immediately decide to use the `check_order_status` tool.

In [ ]:
# Test 1 - Simple Single-Step Task
react_agent("Check the status of order ORD12345");

### Test 1: Simple Order Lookup
#### Iteration 1:
**THOUGHT:** "I need to check the status for ORD12345. I will use the check_order_status tool".  
**ACTION:** Calls the tool.
#### Iteration 2:
**THOUGHT:** "I see the tool returned 'Shipped'. I have enough information now".  
**FINAL ANSWER:** "Your order is shipped...".

#### **The model's output is a `tool_calls` request (finish_reason="tool_calls").**

### 2. Multi-Step Complex Task (Branching Logic)

This is a true stress test for the ReAct pattern. The agent must first retrieve the orders, then filter for those with a "Delivered" status, and finally execute a separate tool call (`process_refund`) for each eligible item.

In [ ]:
# Test 2 - Multi-Step Complex Task
react_agent("Hi, I'm customer CUST003. Show me all my orders and process a refund for any delivered items.");

### 🎓 Deep Dive: What Just Happened?

In Test 3, the agent performed a "Multi-Step" workflow. Unlike a traditional script, we didn't tell the agent to check for delivered items; it deduced that requirement from the user's request and the data it observed.

#### 🔄 The Reasoning Chain

| Step | Phase | Agent's Logic | Action Taken |
|------|-------|---------------|--------------|
| 1 | Discovery | "I need to identify the customer's orders first." | Called `get_customer_orders` |
| 2 | Validation | "I see order `ORD12347` is delivered; it is eligible for a refund." | Called `process_refund` |
| 3 | Resolution | "I have completed all requests. Time to summarize." | Generated final response |

#### 💡 Key Agentic Insights

> [!IMPORTANT]
> **What makes this "Agentic"?**
> 1. **Autonomy:** The agent DECIDES when to call tools and which ones to use based on the context.
> 2. **Evaluation:** The agent DECIDES when the task is complete.
> 3. **Memory:** By maintaining the `messages` history, the agent remembers that it already processed a refund in Iteration 2 so it doesn't repeat it in Iteration 3.

### 3. Advanced Task
This scenario adds a mathematical requirement ("Calculate my total spending") on top of the data retrieval and refund processing. It tests the agent's ability to keep track of a long conversation history and multiple goals simultaneously.

In [ ]:
# Test 3 - A Complex Task
reset_database()

react_agent("""
            I'm customer CUST001. Please:
            1. Show me all my orders
            2. Tell me which ones are delivered
            3. Calculate my total spending
            4. Process refunds for all delivered items
            """);

## 🧪 Experiment: Exploring the Safety Limit
One of the most important parts of an agent loop is the Iteration Limit. Without it, an agent that gets confused could loop infinitely, consuming thousands of tokens. In this experiment, we force the agent to fail by giving it a complex task but only allowing it 2 iterations.

In [ ]:
# Experiment - Limited Iterations
reset_database()

react_agent(
    "I'm CUST001. Show me all my orders and refund any delivered items.", 
    max_iterations=2
);